
# Ch11 — Forecasting 실습 (MLOps 도입 가이드)

이 노트북은 소비(전력) 예측 시나리오를 예제로, **데이터 준비 → 베이스라인 → 모델링(Prophet) → 평가 → MLflow 기록** 까지 한 번에 실습할 수 있도록 설계되었습니다.

## 학습 목표
- 시계열 데이터 준비 및 시각화
- 베이스라인(naive, 이동평균) vs ML 모델 성능 비교
- Prophet을 이용한 예측 및 구성요소 분석
- MLflow를 통한 실험 기록(파라미터/지표/플롯/아티팩트)

> ⚠️ 실제 책의 데이터셋이 공개되지 않았다고 가정하여 **합성 데이터**로 재현합니다.  
> 원한다면 CSV를 불러와 동일한 파이프라인에 대입할 수 있도록 셀을 분리해두었습니다.


In [ ]:

# (선택) 의존성 설치: 로컬 환경에 Prophet, MLflow가 없다면 실행하세요.
# 주피터 환경/OS 따라 설치가 다를 수 있습니다. 실패 시 공식 문서를 참고하세요.
# !pip install -q pandas numpy matplotlib scikit-learn mlflow prophet


In [ ]:

import os
import math
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Prophet은 설치가 안 되어 있으면 아래 import에서 오류가 날 수 있습니다.
try:
    from prophet import Prophet
    HAS_PROPHET = True
except Exception as e:
    HAS_PROPHET = False
    print("Prophet 미설치 또는 로드 실패:", e)

try:
    import mlflow
    HAS_MLFLOW = True
except Exception as e:
    HAS_MLFLOW = False
    print("MLflow 미설치 또는 로드 실패:", e)

np.random.seed(42)
pd.set_option("display.max_rows", 10)



## 1. 데이터 준비 (합성 시계열)
- 일 단위 전력소비량을 2년간 생성
- 주기성(약 30일), 주간 패턴(요일 효과), 트렌드, 잡음 포함


In [ ]:

# 날짜 생성: 2년(2023-01-01 ~ 2024-12-31)
ds = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")

# 기본 수준 + 월간 주기성 + 주간(요일) 패턴 + 점진적 트렌드 + 노이즈
base = 120
monthly_cycle = 10 * np.sin(np.arange(len(ds))/30)
weekday = pd.Series(ds).dt.weekday.values  # 0=Mon ... 6=Sun
weekday_effect = np.where(weekday < 5, 5, -3)  # 평일 ↑, 주말 ↓ (예시)
trend = np.linspace(0, 15, len(ds))           # 완만한 증가
noise = np.random.normal(0, 3, len(ds))

y = base + monthly_cycle + weekday_effect + trend + noise

df = pd.DataFrame({"ds": ds, "y": y})
df.head()



### (선택) CSV 로드로 대체
아래 셀에서 `your_timeseries.csv` 파일을 불러올 수 있습니다.  
열 이름은 `ds`(datetime), `y`(target) 로 맞춰주세요.


In [ ]:

# csv_path = "your_timeseries.csv"
# df = pd.read_csv(csv_path, parse_dates=["ds"])[["ds", "y"]]
# df = df.sort_values("ds").reset_index(drop=True)
# df.head()



## 2. 빠른 EDA (시각화)


In [ ]:

plt.figure(figsize=(12,4))
plt.plot(df["ds"], df["y"], label="usage")
plt.title("Daily Consumption (Synthetic)")
plt.xlabel("date"); plt.ylabel("y")
plt.legend(); plt.show()

print(df.describe())



## 3. 학습/검증 분리
- 최근 30일을 검증 집합으로 사용


In [ ]:

horizon = 30
train = df.iloc[:-horizon].copy()
valid = df.iloc[-horizon:].copy()
print(train.tail(2))
print(valid.head(2))



## 4. 베이스라인 모델
- **Naive(last)**: 마지막 관측값을 그대로 예측
- **Seasonal Naive (last week)**: 7일 전 값을 예측값으로 사용
- **Moving Average(7d)**: 직전 7일 이동평균


In [ ]:

def mae_rmse(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# 4.1 Naive (last value)
last_value = train["y"].iloc[-1]
y_pred_naive = np.array([last_value]*len(valid))
mae_naive, rmse_naive = mae_rmse(valid["y"].values, y_pred_naive)

# 4.2 Seasonal Naive (last week)
# 검증 구간의 각 t에 대해, 학습 마지막 7일 뒤의 값이 없으므로
# 간단히 학습 말미 7일을 valid 길이만큼 순환해 사용
last_week = train["y"].iloc[-7:].values
y_pred_seasonal = np.resize(last_week, len(valid))
mae_seasonal, rmse_seasonal = mae_rmse(valid["y"].values, y_pred_seasonal)

# 4.3 Moving Average (7d)
rolling = train["y"].rolling(7).mean().dropna()
ma7 = rolling.iloc[-1]
y_pred_ma7 = np.array([ma7]*len(valid))
mae_ma7, rmse_ma7 = mae_rmse(valid["y"].values, y_pred_ma7)

pd.DataFrame({
    "model": ["naive_last", "seasonal_naive(last_week)", "moving_avg_7d"],
    "MAE": [mae_naive, mae_seasonal, mae_ma7],
    "RMSE": [rmse_naive, rmse_seasonal, rmse_ma7]
})



## 5. Prophet 모델 학습
- 시계열 전용 알고리즘 Prophet으로 모델링
- 예측 구간: 30일


In [ ]:

if HAS_PROPHET:
    m = Prophet()
    m.fit(train.copy())

    future = m.make_future_dataframe(periods=len(valid), freq="D")
    fcst = m.predict(future)

    # 검증 구간에 해당하는 부분 추출
    fcst_valid = fcst.iloc[-len(valid):][["ds", "yhat", "yhat_lower", "yhat_upper"]].reset_index(drop=True)
    eval_df = valid.reset_index(drop=True).merge(fcst_valid, on="ds", how="left")

    mae_prophet, rmse_prophet = mae_rmse(eval_df["y"].values, eval_df["yhat"].values)

    print("Prophet → MAE:", round(mae_prophet, 4), "RMSE:", round(rmse_prophet, 4))

    # 플롯
    fig1 = m.plot(fcst)
    plt.title("Prophet Forecast")
    plt.show()

    fig2 = m.plot_components(fcst)
    plt.show()
else:
    print("Prophet이 설치되어 있지 않아 이 섹션을 건너뜁니다. 상단 '설치' 셀을 먼저 실행하세요.")



## 6. 성능 비교 (베이스라인 vs Prophet)


In [ ]:

rows = [
    ("naive_last", mae_naive, rmse_naive),
    ("seasonal_naive(last_week)", mae_seasonal, rmse_seasonal),
    ("moving_avg_7d", mae_ma7, rmse_ma7)
]
if HAS_PROPHET:
    rows.append(("prophet", mae_prophet, rmse_prophet))

compare_df = pd.DataFrame(rows, columns=["model", "MAE", "RMSE"]).sort_values("RMSE").reset_index(drop=True)
compare_df



## 7. MLflow 기록
- 모델/파라미터/지표/플롯 저장


In [ ]:

if HAS_MLFLOW:
    mlflow.set_experiment("ch11_forecasting_demo")

    with mlflow.start_run(run_name="baseline_vs_prophet"):
        # 파라미터
        mlflow.log_param("horizon", 30)
        mlflow.log_param("has_prophet", HAS_PROPHET)

        # 지표
        mlflow.log_metric("mae_naive", mae_naive)
        mlflow.log_metric("rmse_naive", rmse_naive)
        mlflow.log_metric("mae_seasonal", mae_seasonal)
        mlflow.log_metric("rmse_seasonal", rmse_seasonal)
        mlflow.log_metric("mae_ma7", mae_ma7)
        mlflow.log_metric("rmse_ma7", rmse_ma7)

        if HAS_PROPHET:
            mlflow.log_metric("mae_prophet", mae_prophet)
            mlflow.log_metric("rmse_prophet", rmse_prophet)

        # 아티팩트(플롯 저장 후 로깅)
        os.makedirs("artifacts", exist_ok=True)
        plt.figure(figsize=(12,4))
        plt.plot(df["ds"], df["y"], label="usage")
        plt.title("Daily Consumption (Synthetic)")
        plt.xlabel("date"); plt.ylabel("y"); plt.legend()
        plt.tight_layout()
        plot_path = "artifacts/timeseries.png"
        plt.savefig(plot_path)
        plt.close()
        mlflow.log_artifact(plot_path)

        # 비교표 저장
        compare_path = "artifacts/compare.csv"
        compare_df.to_csv(compare_path, index=False)
        mlflow.log_artifact(compare_path)

        # Prophet 모델 저장(옵션)
        if HAS_PROPHET:
            # 간단히 pickle 저장 (프로덕션에선 전용 저장 포맷/모듈 권장)
            with open("artifacts/prophet.pkl", "wb") as f:
                pickle.dump(m, f)
            mlflow.log_artifact("artifacts/prophet.pkl")

    print("MLflow 로깅 완료. `mlflow ui` 로 대시보드를 확인하세요.")
else:
    print("MLflow 미설치로 기록을 건너뜁니다. 상단 '설치' 셀을 먼저 실행하세요.")



## 8. (옵션) 서빙 아이디어
FastAPI로 간단한 예측 API를 구성할 수 있습니다. 아래는 **개념 예시**입니다.

```python
# app.py
from fastapi import FastAPI
import pandas as pd
import pickle
from prophet import Prophet

app = FastAPI()
model = pickle.load(open("artifacts/prophet.pkl", "rb"))

@app.get("/forecast")
def forecast(days: int = 7):
    future = model.make_future_dataframe(periods=days, freq="D")
    fcst = model.predict(future).tail(days)[["ds", "yhat", "yhat_lower", "yhat_upper"]]
    return fcst.to_dict(orient="records")
```
